# <center>OCR Project To Detect Sanskrit Lines From A Pdf File</center>

## Summary For The Notebook :

* Converts Pdf Pages Into A List Of Images.
* Scans The Images.
* Detects For Possible Sanskrit Words.
* Creates A Bounding Box Around The Detected Word.
* Merges The Boundig Boxes Which Are In The Line Of Text
* Captures The Box And Saves It As An Image File With Extension ".jpg".
* Saves The Coordinate's Of Boxes In A Json File.
* The Box Name In The Json File Is As Follows : [box_"page-no"_"image-no"]

### Prerequisite's 
1. An Empty Json File Named "box_coordinates.json"
2. An Empty Folder Named "/box_imgs"
3. An Empty Folder Named "/pdf_imgs"

### Note :
    Make Changes In The Notebook For URL Path's Acording To Your System

### Necessary Imports

In [54]:
import pytesseract
import cv2
import os
import json
import glob
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
from tqdm import tqdm

### Pdf To Image Conversion

In [55]:
pytesseract.pytesseract.tesseract_cmd=r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
images = convert_from_path("OCR_SANSKRIT/Sanskrit_Text.pdf",100)
path = 'C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/pdf_imgs/'

In [56]:
for i in range(len(images)):   
      # Save pages as images in the pdf
    images[i].save(os.path.join(path ,'page'+ str(i) +'.jpg'), 'JPEG')

### Saving Images In List Format

In [57]:
img_list = list(map(cv2.imread,glob.glob('{}*.jpg'.format(path))))

### Dumping Data To Json File

In [58]:
def write_json(data, filename='C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/box_coordinates.json'):
    with open(filename,'w') as f:
        json.dump(data, f, indent=4)

### Appending Data To Json File

In [59]:
def box_details(name,tl,tr,bl,br):
    with open('C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/box_coordinates.json', mode='r', encoding='utf-8') as jf:
        data = json.load(jf)
        temp = data['Bounding_Boxes']
        # python object to be appended
        y = {'name': name, 'top_left': tl ,'top_right': tr ,'bottom_left': bl , 'bottom_right': br}
        # appending data to 'Bounding_Boxes'
        temp.append(y)
    write_json(data)

### Function For Extracting Merged Bounding Box

In [60]:
def extract_box(img,x,y,w,h,counter,ROI_number):
    original = img.copy()
    ROI = original[y:y+h, x:x+w]
    cv2.imwrite('C:/Users/tejas/OneDrive/Documents/GitHub/OCR_SANSKRIT/box_imgs/box_{}_{}.jpg'.format(counter,ROI_number), ROI)    

### Function For Merging Bounding Boxes

In [61]:
def merge_box(rList,counter,ROI_number):
    arr = []
    for x,y,w,h in rList:
        arr.append((x,y))
        arr.append((x+w,y+h))
    x,y,w,h = cv2.boundingRect(np.asarray(arr))
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),1)
    name = 'box_{}_{}'.format(counter,ROI_number)
    tl = '[{},{}]'.format(x , y)
    tr = '[{},{}]'.format(x+w , y)
    bl = '[{},{}]'.format(x , y+h)
    br = '[{},{}]'.format(x+w , y+h)
    extract_box(img,x,y,w,h,counter,ROI_number)
    box_details(name,tl,tr,bl,br)

### Code For Generating Bounding Boxes

In [62]:
counter = 1
for i in tqdm(range(len(img_list))):
    img = img_list[i]
    img = cv2.resize(img, None,fx=0.8,fy=0.8)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    kernel_sharpening = np.array([[-1,-1,-1],
                                  [-1,9,-1],
                                  [-1,-1,-1],])
    img = cv2.filter2D(img,-1, kernel_sharpening)
    boxes = pytesseract.image_to_data(img,lang = 'hin')
    rList = []
    ROI_number = 0
    for x,b in enumerate(boxes.splitlines()):
        if x!=0 :
            b = b.split()
            if int(b[5]) > 0:
                x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                rList.append([x,y,w,h])
            elif len(rList)==0:
                continue
            else:
                ROI_number+=1
                merge_box(rList,counter,ROI_number)
                rList.clear()
    counter = counter + 1

100%|██████████| 5/5 [00:23<00:00,  4.73s/it]


   #### Once The Prgress Bar Is Completed Check For The Data In Respective Files

####  Thank You For Using :) 